## Анализ базы данных с помощью SQL-запросов

задача — проанализировать базу данных крупного сервиса для чтения книг по подписке.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def table_info(table):
    """Display information about the table"""
    
    print('Первые 4 строки таблицы:')
    display(pd.io.sql.read_sql('''SELECT *
                                  FROM {} 
                                  LIMIT 4'''.format(table), con = engine))

In [3]:
table_info('books')

Первые 4 строки таблицы:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309


In [4]:
table_info('publishers')

Первые 4 строки таблицы:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover


In [5]:
table_info('ratings')

Первые 4 строки таблицы:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3


In [6]:
table_info('reviews')

Первые 4 строки таблицы:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...


__Посчитайте, сколько книг вышло после 1 января 2000 года__

In [7]:
query = '''SELECT COUNT(book_id) as count_books
           FROM books
           WHERE publication_date >= '2000-01-01';
        '''
pd.io.sql.read_sql(query, con = engine)

,count_books
0,821


__Для каждой книги посчитайте количество обзоров и среднюю оценкy__

In [8]:
query = '''SELECT b.title AS title,
                  COUNT(DISTINCT rev.review_id) as count_reviews,
                  AVG(rat.rating) as avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY title
           ORDER BY count_reviews DESC;
        '''
pd.io.sql.read_sql(query, con = engine)

,title,count_reviews,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


In [13]:
query = '''SELECT b.title AS title,
                  COUNT(DISTINCT rev.review_id) as count_reviews,
                  AVG(rat.rating) as avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
           GROUP BY title
           ORDER BY avg_rating DESC
           LIMIT 30;
        '''
pd.io.sql.read_sql(query, con = engine)

,title,count_reviews,avg_rating
0,School's Out—Forever (Maximum Ride #2),3,5.0
1,Pop Goes the Weasel (Alex Cross #5),2,5.0
2,Plum Island (John Corey #1),2,5.0
3,Evil Under the Sun (Hercule Poirot #24),2,5.0
4,Moneyball: The Art of Winning an Unfair Game,3,5.0
5,Piercing the Darkness (Darkness #2),2,5.0
6,Light in August,2,5.0
7,March,2,5.0
8,Emily of New Moon (Emily #1),2,5.0
9,Evening Class,2,5.0


__Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры__

In [16]:
query = '''SELECT p.publisher AS publisher,
                  COUNT(b.book_id) as count_books
           FROM publishers AS p
           LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50
           GROUP BY publisher
           ORDER BY count_books DESC
           LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


__Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками__

In [14]:
query = '''SELECT author, 
                  AVG(avg_rating) AS avg_rating_author
           FROM (SELECT author_id, 
                        title, 
                        AVG(rating) AS avg_rating
                 FROM books AS b
                 LEFT JOIN ratings AS rat ON b.book_id = rat.book_id 
                 GROUP BY title, author_id
                 HAVING COUNT(rating_id) >= 50) AS avg
           LEFT JOIN authors AS a ON avg.author_id = a.author_id
           GROUP BY author
           ORDER BY avg_rating_author DESC
           LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating_author
0,J.K. Rowling/Mary GrandPré,4.283844


__Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок__

In [15]:
query = '''SELECT ROUND(AVG(count_reviews)) AS avg_reviews
           FROM (SELECT COUNT(review_id) AS count_reviews
                 FROM (SELECT username
                       FROM ratings AS rat
                       GROUP BY username
                       HAVING COUNT(rating) > 50) AS rat
                 LEFT JOIN reviews AS rev ON rat.username = rev.username
                 GROUP BY rat.username) AS rev;
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.0


## Выводы

- после 1 января 2000 года вышла 821 книга
- по количеству обзоров лидирует книга "мемуары Гейши" - 8 обзоров, второе место Сумерки -7, и Гарри Поттер и тайная комната - 6
- более 30 книг имеют среднюю оценку 5.0, но книга у которой самый высокий рейтинг и есть более 5 обзоров - это Гарри Поттер. оценка 4.29 и 6 отзывов
- издательство, которое выпустило наибольшее число книг толще 50 страниц - это Penguin Books, выпустила 42 книги
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 обзора
- автор с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками - J.K. Rowling/Mary GrandPré	